# Electric Vehicle Population - Spark Structured Streaming
This notebook consumes EV data from Kafka and performs streaming analytics.

In [1]:
pip install kafka-python pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import time
import uuid
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, count, desc
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__} pyspark-shell'
os.environ['SPARK_SUBMIT_OPTS'] = '-Djdk.security.auth.login.Config=ignore'

In [4]:
KAFKA_BROKER_URL = "localhost:9092"
KAFKA_TOPIC = "electric-cars"

In [5]:
# Create SparkSession
# Note: Using a unique checkpoint dir to avoid stale checkpoint issues
import uuid
checkpoint_dir = f"./checkpoint_ev_{uuid.uuid4().hex[:8]}"

spark = SparkSession.builder \
    .appName("ElectricVehicles-Streaming") \
    .config("spark.jars.packages", f"org.apache.spark:spark-sql-kafka-0-10_2.13:{pyspark.__version__}") \
    .config("spark.sql.streaming.checkpointLocation", checkpoint_dir) \
    .getOrCreate()

print(f"Checkpoint location: {checkpoint_dir}")

Checkpoint location: ./checkpoint_ev_9cdb878d


In [6]:
# Schema matching the cleaned data from producer.py
ev_schema = StructType([
    StructField("vin", StringType(), True),
    StructField("county", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("make", StringType(), True),
    StructField("model", StringType(), True),
    StructField("electric_vehicle_type", StringType(), True),
    StructField("cafv_eligibility", StringType(), True),
    StructField("electric_utility", StringType(), True),
    StructField("model_year", IntegerType(), True),
    StructField("electric_range", IntegerType(), True),
    StructField("base_msrp", IntegerType(), True),
    StructField("legislative_district", IntegerType(), True),
    StructField("dol_vehicle_id", IntegerType(), True),
    StructField("census_tract", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("is_bev", BooleanType(), True),
    StructField("range_category", StringType(), True),
    StructField("is_cafv_eligible", BooleanType(), True)
])

In [7]:
# DEBUG: First read raw Kafka data (batch mode) to verify messages exist
# This reads WITHOUT streaming to check if topic has data
try:
    raw_batch = spark.read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BROKER_URL) \
        .option("subscribe", KAFKA_TOPIC) \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()
    
    print(f"Total messages in Kafka topic: {raw_batch.count()}")
    print("\nSample raw messages:")
    raw_batch.selectExpr("CAST(value AS STRING) as raw_json").show(5, truncate=False)
except Exception as e:
    print(f"Error reading from Kafka: {e}")
    print("Make sure Kafka is running and the topic exists!")

Total messages in Kafka topic: 633845

Sample raw messages:
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|raw_json                                                                                                                                                                                                                                                                                                                                 

## 2. Read and Display EV Data
Parse the Kafka messages and display the data.

In [8]:
# Parse the raw batch data into structured DataFrame
parsed_df = raw_batch.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), ev_schema).alias("data")) \
    .select("data.*")

# Show sample data
print(f"Total parsed records: {parsed_df.count()}")
parsed_df.show(10, truncate=False)

Total parsed records: 633845
+----------+---------+-----------+-----+-----------+--------+--------------+--------------------------------------+------------------------------------------------------------+---------------------------------------------+----------+--------------+---------+--------------------+--------------+------------+----------+------------+------+--------------+----------------+
|vin       |county   |city       |state|postal_code|make    |model         |electric_vehicle_type                 |cafv_eligibility                                            |electric_utility                             |model_year|electric_range|base_msrp|legislative_district|dol_vehicle_id|census_tract|latitude  |longitude   |is_bev|range_category|is_cafv_eligible|
+----------+---------+-----------+-----+-----------+--------+--------------+--------------------------------------+------------------------------------------------------------+---------------------------------------------+-------

## 3. Top 3 Cities with Most Electric Cars in 2023
Filter for model year 2023 and aggregate by city.

In [9]:
# Create temporary view for SQL queries
parsed_df.createOrReplaceTempView("electric_vehicles")

# Top 3 cities with most electric cars in 2023 using SQL
print("Top 3 cities with most electric cars in 2023:")
spark.sql("""
    SELECT city, COUNT(*) as ev_count
    FROM electric_vehicles
    WHERE model_year = 2023
    GROUP BY city
    ORDER BY ev_count DESC
    LIMIT 3
""").show(truncate=False)

Top 3 cities with most electric cars in 2023:
+--------+--------+
|city    |ev_count|
+--------+--------+
|Seattle |23726   |
|Bellevue|9181    |
|Tukwila |8375    |
+--------+--------+

